# Recuperar informações da Wikipedia

In [ ]:
import requests

In [ ]:
req = requests.get("https://pt.wikipedia.org/wiki/Unidades_federativas_do_Brasil")

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
bs = BeautifulSoup(req.text, "html.parser")

In [ ]:
print(bs.prettify())

In [ ]:
print(bs.h1.string)

In [ ]:
bs.h1

## Recuperar todos os links disponíveis nessa página

In [ ]:
bs.a

Note que apenas um único link foi retornado. Para retornar todos precisamos utilizar um método chamado ```findall()```.

In [ ]:
links = bs.findAll('a')
print(links)

In [ ]:
print(type(links))

Note que além dos links, tem outras informações.

In [ ]:
links[3]

Para imprimir/salvar apenas os links, precisamos iterar na lista que foi criada para pegar apenas o conteúdo do href.

In [ ]:
for link in links:
    print(link.get("href"))

É importante verificar que estamos trazendo todos os links (internos e externos) com essa abordagem.

## Recuperar uma tabela

Como vimos, iremos recuperar a tabela que está disponível na página, que lista todos os estados do Brasil, adicionando outras informações, como a abreviação, capital, area, população, mortalidade infantil, entre outras.

Existem duas formas de recuperarmos a tabela correta. A primeira, seria recuperar todas as tabelas disponível, iterar em cada uma delas e verificar a classe que ela pertecem.

A outra forma, é mais rápida e simples. No navegador Chrome, abrar a página em questão, clique com o Botão Direito em cima da tabela que deseja recuperar e depois clique em Inspecionar (Ctrl + Shift + I) e copie o nome da classe.

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center; font-size:85%">
```

Iremos utilizar ```wikitable sortable jquery-tablesorter``` para recuperar somente essa tabela.

In [ ]:
tabela = bs.findAll('table', 
                 class_='wikitable sortable')

In [ ]:
#print(len(tabela))
#tabela

Ótimo! Já temos a tabela recuperada.

O que iremos fazer agora é extrair as informações e adicionar em um DataFrame. Para isso precisamos iterar em cada linha (tr) e então atribuir cada elemento de tr (td) à uma variável e então adicionar a uma lista.

In [ ]:
colunas = []
cabecalho = tabela[0].findAll('tr')[0].findAll('th')
for item in cabecalho:
    if not item.find('a'):
        colunas.append(item.string)
    else:
        colunas.append(item.a.string)

In [ ]:
colunas[colunas.index('2005')]='Densidade'

In [ ]:
colunas[colunas.index('2003')]='Alfabetização'
colunas[colunas.index('2010')]='Expectativa de vida'
colunas

In [ ]:
linha = tabela[0].findAll('tr')[1].findAll('td')

In [ ]:
linha[0].find('a').get('href')

In [ ]:
linha[13].find(text=True)

In [ ]:
dados = {
    'A' : [],
    'B' : [],
    'C' : [],
    'D' : [],
    'E' : [],
    'F' : [],
    'G' : [],
    'H' : [],
    'I' : [],
    'J' : [],
    'K' : [],
    'L' : [],
    'M' : [],
    'N' : []
}

In [ ]:
linhas = tabela[0].findAll('tr')
for l in range(1, len(linhas)):
    linha = tabela[0].findAll('tr')[l].findAll('td')
    dados['A'].append(linha[0].find('a').get('href'))
    dados['B'].append(linha[1].find(text=True))
    dados['C'].append(linha[2].find(text=True))
    dados['D'].append(linha[3].find(text=True))
    dados['E'].append(linha[4].find(text=True))
    dados['F'].append(linha[5].find(text=True))
    dados['G'].append(linha[6].find(text=True))
    dados['H'].append(linha[7].find(text=True))
    dados['I'].append(linha[8].find(text=True))
    dados['J'].append(linha[9].find(text=True))
    dados['K'].append(linha[10].find(text=True))
    dados['L'].append(linha[11].find(text=True))
    dados['M'].append(linha[12].find(text=True))
    dados['N'].append(linha[13].find(text=True))

In [ ]:
len(dados['L'])

In [ ]:
print(dados)

Agora, basta criar o DataFrame

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(columns=colunas)
df

In [ ]:
lista = list(dados.keys())
lista.sort()
print(lista)

In [ ]:
for i in range(0, len(colunas)):
    df[colunas[i]] = dados[lista[i]]

In [ ]:
df.head()

In [ ]:
df_final = df.drop('Bandeira', axis=1)

In [ ]:
df_final

### Realizando ajustes e conversões de tipos
Antes de converter para float ou int, precisamos renomear algumas colunas para facilitar o acesso.

Primeiro criamos uma lista com os nomes alterados:

In [ ]:
colunas_final = []
for col in df_final.columns:
    col = col.title()
    for c in ['(', ')', ' ', '%']:
        if c == '%':
            col = col.replace(c, 'Percentual')
        else:
            col = col.replace(c, '')
    colunas_final.append(col)
print(colunas_final)

É necessário alterar os nomes das colunas no ```df_final```.

In [ ]:
for i in range(0, len(df_final.columns)):
    df_final.rename(columns={
        df_final.columns[i] : colunas_final[i]
    }, inplace=True)

In [ ]:
df_final.rename(columns={
        'Alfabetização' : 'AlfabetizaçãoPercentual',
        'MortalidadeInfantil' : 'MortalidadeInfantilPercentual'
    }, inplace=True)

Também será necessário criar duas listas auxiliares com o nome das colunas que iremos converter para float e int.

In [ ]:
colunas_float = ['Área', 'Densidade', 'PercentualTotal', 'Idh', 
                 'MortalidadeInfantilPercentual', 'ExpectativaDeVida']

colunas_int = ['População', 'Pib', 'PibPerCapita', 'AlfabetizaçãoPercentual']

Realizando uma copia do df_final para tratamento.

In [ ]:
df_aux = df_final.copy()

Funções auxiliares para remover espaços e converter a pontuação.

In [ ]:
def remover_espacos(string):
    string = string.replace(u'%', '')
    string = string.replace(u'‰', '')
    return string.replace(u'\xa0', '')

In [ ]:
def converter_pontuacao(string):
    return string.replace(u',', '.')

Aplicando as funções criadas em cada uma das colunas definidas na variável ```colunas_float```.

In [ ]:
aux_float = df_final[colunas_float].applymap(remover_espacos)
aux_float = aux_float[colunas_float].applymap(converter_pontuacao)
aux_float[:5]

Aplicando as funções criadas em cada uma das colunas definidas na variável ```colunas_int```.

In [ ]:
aux_int = df_final[colunas_int].applymap(remover_espacos)
aux_int = aux_int[colunas_int].applymap(converter_pontuacao)
aux_int[:5]

Criar função para converter o tipo da coluna.

In [ ]:
def converter_tipo(df, colunas, tipo):
    return df[list(colunas)].astype(tipo)

Realizando a conversão de tipo para int e float para as respectivas colunas.

In [ ]:
df_int = converter_tipo(aux_int, aux_int.columns, int)
df_float = converter_tipo(aux_float, aux_float.columns, float)

Aplicando os valores convertidos no DataFrame final.

In [ ]:
#float
df_final['Área'] = df_float['Área']
df_final['Densidade'] = df_float['Densidade']
df_final['PercentualTotal'] = df_float['PercentualTotal']
df_final['Idh'] = df_float['Idh']
df_final['MortalidadeInfantilPercentual'] = df_float['MortalidadeInfantilPercentual']
df_final['ExpectativaDeVida'] = df_float['ExpectativaDeVida']
# Int
df_final['População'] = df_int['População']
df_final['Pib'] = df_int['Pib']
df_final['PibPerCapita'] = df_int['PibPerCapita']
df_final['AlfabetizaçãoPercentual'] = df_int['AlfabetizaçãoPercentual']

In [ ]:
df_final